In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as tkr
import seaborn as sns
import scipy as stats

import nltk # sentiment library
nltk.download('vader_lexicon') # download vader lexicon
from nltk.sentiment import SentimentIntensityAnalyzer as SIA

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kevinvo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [68]:
#provides relative paths for csv as referenced from a folder
import os
for dirname, _, filenames in os.walk('data'): #path to folder
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/.DS_Store
data/stocks/Company.csv
data/stocks/CompanyValues.csv
data/tweets/Company.csv
data/tweets/Company_Tweet.csv
data/tweets/Tweet.csv


In [69]:
# read CSV and import tweets
tweets=pd.read_csv('data/tweets/Tweet.csv')
company_tweets=pd.read_csv('data/tweets/Company_Tweet.csv')
tweets=tweets.merge(company_tweets,how='left',on='tweet_id')

# add time format dates 
tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
tweets.date=pd.to_datetime( tweets.date,errors='coerce')
tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time
#1420070457 to 2015-01-01 00:00:57

In [70]:
len(tweets)
len(company_tweets)
type(tweets)
tweets['body'][100]
tweets['time'].head()
company_tweets.head()

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL


In [91]:
# SentimentIntensityAnalyzer is a class. 
# You need to initialize an object of SentimentIntensityAnalyzer and call the polarity_scores() method on that.
sia = SIA()
testSentence = "Tesla revenue ATH buy! all time high! =) cool"
print(testSentence)
sia.polarity_scores(testSentence)

Tesla revenue ATH buy! all time high! =) cool


{'neg': 0.0, 'neu': 0.535, 'pos': 0.465, 'compound': 0.7256}

In [92]:
# add to lexicon
pos_csv_str='buy,bull,long,support,undervalued,underpriced,cheap,upward,rising,trend,moon,rocket,hold,breakout,call,beat,support,buying,holding,high,profit,ATH,all time high, =), cool'
neg_csv_str='sell,bear,bubble,bearish,short,overvalued,overbought,overpriced,expensive,downward,falling,sold,sell,low,put,miss,resistance,squeeze,cover,seller'

dictPositive = { i : 4 for i in pos_csv_str.split(",") }
dictNegative = { i : -4 for i in neg_csv_str.split(",") }
add_lexicon = {**dictPositive, **dictNegative}
# print(Financial_Lexicon)

In [93]:
sia.lexicon.update(add_lexicon)
print(testSentence)
sia.polarity_scores(testSentence)

Tesla revenue ATH buy! all time high! =) cool


{'neg': 0.0, 'neu': 0.237, 'pos': 0.763, 'compound': 0.9523}